In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
218,AFG,Asia,Afghanistan,2020-08-15,37431.0,7.0,1363.0,0.0,961.536,0.180,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
378,ALB,Europe,Albania,2020-08-15,7117.0,146.0,219.0,6.0,2473.070,50.733,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
602,DZA,Africa,Algeria,2020-08-15,37664.0,477.0,1351.0,10.0,858.908,10.878,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-14,France,211344.0,0.0,30425.0,0.0,0.0,0.0
0,2020-08-14,China,89222.0,0.0,4707.0,0.0,0.0,0.0
0,2020-08-14,Italy,252766.0,0.0,35262.0,0.0,0.0,0.0
0,2020-08-14,Spain,339994.0,0.0,28615.0,0.0,0.0,0.0
0,2020-08-14,United States,5334859.0,0.0,168534.0,0.0,0.0,0.0
0,2020-08-14,World,21291617.0,0.0,765466.0,0.0,0.0,0.0
0,2020-08-14,United Kingdom,315028.0,0.0,46904.0,0.0,0.0,0.0
0,2020-08-14,Germany,222711.0,0.0,9235.0,0.0,0.0,0.0
0,2020-08-14,Iran,340016.0,0.0,19401.0,0.0,0.0,0.0
0,2020-08-14,Turkey,246702.0,0.0,5928.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-15,France,211344.0,212211.0,30425.0,30406.0,-867.0,19.0
0,2020-08-15,China,89222.0,89214.0,4707.0,4701.0,8.0,6.0
0,2020-08-15,Italy,252766.0,252809.0,35262.0,35234.0,-43.0,28.0
0,2020-08-14,Spain,339994.0,342813.0,28615.0,28617.0,-2819.0,-2.0
0,2020-08-15,United States,5334859.0,5313080.0,168534.0,168446.0,21779.0,88.0
0,2020-08-15,World,21291617.0,21213649.0,765466.0,760421.0,77968.0,5045.0
0,2020-08-15,United Kingdom,315028.0,316367.0,46904.0,41357.0,-1339.0,5547.0
0,2020-08-15,Germany,222711.0,222828.0,9235.0,9231.0,-117.0,4.0
0,2020-08-15,Iran,340016.0,338825.0,19401.0,19331.0,1191.0,70.0
0,2020-08-15,Turkey,246702.0,246861.0,5928.0,5934.0,-159.0,-6.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")